In [32]:
import numpy as np
def tanh(z):
	return (np.exp(z) - np.exp(-z)) / (np.exp(z) + np.exp(-z))

class mlp:
    """A Multi-Layer Perceptron"""

    def __init__(
        self, inputs, targets, nhidden, beta=1, momentum=0.9, outtype="logistic"
    ):
        """Constructor"""
        self.nin = np.shape(inputs)[1]
        self.nout = np.shape(targets)[1]
        self.ndata = np.shape(inputs)[0]
        self.nhidden = nhidden

        self.beta = beta
        self.momentum = momentum
        self.outtype = outtype

        # Initialise network
        self.weights1 = (
            (np.random.rand(self.nin + 1, self.nhidden) - 0.5) * 2 / np.sqrt(self.nin)
        )
        self.weights2 = (
            (np.random.rand(self.nhidden + 1, self.nout) - 0.5)
            * 2
            / np.sqrt(self.nhidden)
        )

    def earlystopping(
        self, inputs, targets, valid, validtargets, eta, niterations=10000
    ):

        valid = np.concatenate((valid, -np.ones((np.shape(valid)[0], 1))), axis=1)

        old_val_error1 = 100002
        old_val_error2 = 100001
        new_val_error = 100000

        print("No. of neurons in hidden layers = ", self.nhidden)
        while ((old_val_error1 - new_val_error) > 0.001) or (
            (old_val_error2 - old_val_error1) > 0.001
        ):
            self.mlptrain(inputs, targets, eta, niterations)
            old_val_error2 = old_val_error1
            old_val_error1 = new_val_error
            validout = self.mlpfwd(valid)
            new_val_error = 0.5 * np.sum((validtargets - validout) ** 2)

        print("Stopped, error = ", new_val_error)
        return new_val_error

    def mlptrain(self, inputs, targets, eta, niterations):
        """Train the neural network"""
        # Add the inputs that match the bias node
        inputs = np.concatenate((inputs, -np.ones((self.ndata, 1))), axis=1)
        change = range(self.ndata)

        updatew1 = np.zeros((np.shape(self.weights1)))
        updatew2 = np.zeros((np.shape(self.weights2)))

        for n in range(niterations):

            self.outputs = self.mlpfwd(inputs)

            error = 0.5 * np.sum((self.outputs - targets) ** 2)

            # Different types of output neurons and their activation functions
            if self.outtype == "linear":
                deltao = (self.outputs - targets) / self.ndata
            elif self.outtype == "logistic":
                deltao = (
                    self.beta
                    * (self.outputs - targets)
                    * self.outputs
                    * (1.0 - self.outputs)
                )
            elif self.outtype == "softmax":
                deltao = (
                    (self.outputs - targets)
                    * (self.outputs * (-self.outputs) + self.outputs)
                    / self.ndata
                )
            elif self.outtype == "tanh":
                deltao = (
                    (self.outputs - targets)
                    * (1.0 - np.power(self.outputs, 2))
                )
            else:
                print("error")

            # hidden network delta
            deltah = (
                self.hidden
                * self.beta
                * (1.0 - self.hidden)
                * (np.dot(deltao, np.transpose(self.weights2)))
            )

            updatew1 = (
                eta * (np.dot(np.transpose(inputs), deltah[:, :-1]))
                + self.momentum * updatew1
            )
            updatew2 = (
                eta * (np.dot(np.transpose(self.hidden), deltao))
                + self.momentum * updatew2
            )
            self.weights1 -= updatew1
            self.weights2 -= updatew2

            return error

    def mlpfwd(self, inputs):
        """Run the network forward"""

        self.hidden = np.dot(inputs, self.weights1)
        self.hidden = 1.0 / (1.0 + np.exp(-self.beta * self.hidden))
        self.hidden = np.concatenate(
            (self.hidden, -np.ones((np.shape(inputs)[0], 1))), axis=1
        )

        outputs = np.dot(self.hidden, self.weights2)

        # Different types of output neurons
        if self.outtype == "linear":
            return outputs
        elif self.outtype == "logistic":
            return 1.0 / (1.0 + np.exp(-self.beta * outputs))
        elif self.outtype == "softmax":
            normalisers = np.sum(np.exp(outputs), axis=1) * np.ones(
                (1, np.shape(outputs)[0])
            )
            return np.transpose(np.transpose(np.exp(outputs)) / normalisers)
        elif self.outtype == "tanh":
            return tanh(outputs)
        else:
            print("error")


    def confmat(self, inputs, targets):
        """Confusion matrix"""

        # Add the inputs that match the bias node
        inputs = np.concatenate((inputs, -np.ones((np.shape(inputs)[0], 1))), axis=1)
        outputs = self.mlpfwd(inputs)

        nclasses = np.shape(targets)[1]

        if nclasses == 1:
            nclasses = 2
            outputs = np.where(outputs > 0.5, 1, 0)
        else:
            # 1-of-N encoding
            outputs = np.argmax(outputs, 1)
            targets = np.argmax(targets, 1)

        cm = np.zeros((nclasses, nclasses))
        for i in range(nclasses):
            for j in range(nclasses):
                cm[i, j] = np.sum(
                    np.where(outputs == i, 1, 0) * np.where(targets == j, 1, 0)
                )
        output = cm
        print("Percentage Correct: ", np.trace(cm) / np.sum(cm) * 100)
        return output


In [33]:
import pandas as pd
from sklearn.neural_network import MLPRegressor
from sklearn.metrics import r2_score
df1 = pd.read_csv('Real Estate.csv')

data1 = df1.drop(['Y house price of unit area'], axis = 1)
target1 = df1['Y house price of unit area']

from sklearn.model_selection import train_test_split
datasets = train_test_split(data1, target1,
                            test_size=0.3)

X_train, X_test, y_train, y_test = datasets


In [38]:
X_train_val = X_train.to_numpy()
y_train_val = y_train.to_numpy().flatten()
X_test_val = X_test.to_numpy()
Y_test_val = y_test.to_numpy().flatten()


In [43]:
t =y_train_val.flatten()
t.shape

(289,)

In [39]:
net = mlp(X_train_val, y_train_val, 25, outtype='softmax')

IndexError: tuple index out of range

In [ ]:
net.mlptrain(X_train_val, y_train_val, 0.001,20000)

<ipython-input-22-fc0eca3559ab>:118: RuntimeWarning: overflow encountered in exp
  self.hidden = 1.0 / (1.0 + np.exp(-self.beta * self.hidden))


68981339.9142391

In [ ]:
test = np.concatenate((X_test_val,-np.ones((np.shape(X_test_val)[0],1))),axis=1)


In [ ]:
out=net.mlpfwd(test)

<ipython-input-22-fc0eca3559ab>:118: RuntimeWarning: overflow encountered in exp
  self.hidden = 1.0 / (1.0 + np.exp(-self.beta * self.hidden))


In [ ]:
out.shape

(125, 289)

In [ ]:
print (0.5*sum((out-Y_test_val)**2))

print (out)
print (out.mean(axis=0))
print (out.var(axis=0))
print (out.max(axis=0))
print (out.min(axis=0))

ValueError: operands could not be broadcast together with shapes (125,289) (125,) 